In [1]:
import matplotlib.pyplot as plt
import pandas as pd

plt.style.use('seaborn-whitegrid')
import numpy as np

In [2]:
folder ='C:XXpathtothefolderXX/'
file='datadocs.csv'
#file2='datadocs2.csv'

In [3]:
dfa = pd.read_csv(folder+file)

In [4]:
#makesure dataframe has no duplicates and time-columns are datetime format
dfa=dfa.drop_duplicates(subset=['tweet_id'])
dfa.tweet_created_at = pd.to_datetime(dfa['tweet_created_at'])

# Clean Retweets

In [ ]:
#eliminate retweets
dfNORT=dfa[~dfa.text.str.contains("RT @")]
#and check the difference
print("total of tweets = "+str(len(dfa))+" Without RT= "+ str(len(dfNORT)))

In [ ]:
print(dfNORT.dtypes)

# Sentiment Analysis of tweets in English

In [7]:
EN=['en']
dfNORTtextcleanEN=dfNORT[dfNORT['lang'].isin(EN)]

Sentiment analysis with Textblob

In [9]:
#pip install textblob

In [10]:
from textblob import TextBlob

In [ ]:
dfNORTtextcleanEN['sentiment'] = dfNORTtextcleanEN['text'].apply(lambda tweet: TextBlob(tweet).sentiment)
#df['language'] = df['text'].apply(lambda tweet: TextBlob(tweet).detect_language())

In [12]:
dfNORTtextcleanENsentiment_series =dfNORTtextcleanEN['sentiment'].tolist()

In [13]:
columns = ['polarity', 'subjectivity']
dfsent = pd.DataFrame(dfNORTtextcleanENsentiment_series, columns=columns, index=dfNORTtextcleanEN.index)

In [14]:
dfNORTtextcleanENSENT=pd.concat([dfNORTtextcleanEN, dfsent], axis=1)

In [ ]:
dfNORTtextcleanENSENT.dtypes

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
data = dfNORTtextcleanENSENT['polarity']
figsize=(20,10)
#N, bins, patches = ax.hist(data, bins=100, edgecolor='white', linewidth=1)
ax.set_ylabel("Number of tweets", fontsize=14)
ax.set_xlabel("Polarity of tweets (negative <----> positive)", fontsize=14)
ax.set_title('Tweet Polarity visualisation',fontsize=18)
N, bins, bars = ax.hist(data, bins=100, color="skyblue", edgecolor='white', linewidth=1)
for bar in bars:
    if bar.get_x() < 0:
        bar.set_facecolor("lightcoral")
plt.savefig(folder+'polarity'+file+'.pdf')

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
data = dfNORTtextcleanENSENT['subjectivity']
figsize=(20,10)
ax.set_ylabel("Number of tweets", fontsize=14)
ax.set_xlabel("Subjectivity of tweets (factual <----> subjective)", fontsize=14)
ax.set_title('Tweet Subjectivity visualisation',fontsize=18)
N, bins, bars = ax.hist(data, bins=100, color="grey", edgecolor='white',  density=True, linewidth=1)
bin_centers = 0.5*(bins[:-1]+bins[1:])
cm = plt.cm.get_cmap("copper")
col = bin_centers - max(bin_centers)
col /= min(col)

for c, p in zip(col, bars):
    plt.setp(p, "facecolor", cm(c))

#for useful colors check this:
#https://matplotlib.org/stable/tutorials/colors/colormaps.html

plt.savefig(folder+'Subjectivity'+file+'.pdf')

Sentiment analysis with VADER

In [9]:
#More about Vader (can do with multiple languages)
#https://python.plainenglish.io/twitter-sentiment-analysis-using-vader-tweepy-b2a62fba151e

In [ ]:
#pip install vaderSentiment

In [8]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
import re


In [9]:
#function obtained here: https://python.plainenglish.io/twitter-sentiment-analysis-using-vader-tweepy-b2a62fba151e

def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)        
    return input_txt
def clean_tweets(tweets):
    #remove twitter Return handles (RT @xxx:)
    tweets = np.vectorize(remove_pattern)(tweets, "RT @[\w]*:") 
    
    #remove twitter handles (@xxx)
    tweets = np.vectorize(remove_pattern)(tweets, "@[\w]*")
    
    #remove URL links (httpxxx)
    tweets = np.vectorize(remove_pattern)(tweets, "https?://[A-Za-z0-9./]*")
    
    #remove special characters, numbers, punctuations (except for #)
    tweets = np.core.defchararray.replace(tweets, "[^a-zA-Z]", " ")
    
    return tweets

In [ ]:
dfNORTtextcleanEN['Newtext'] = clean_tweets(dfNORTtextcleanEN['text'])
dfNORTtextcleanEN.head()

In [11]:
#REF for this: https://github.com/sidneykung/twitter_hate_speech_detection/blob/master/preprocessing/VADER_sentiment.ipynb
# function to calculate polarity scores
pol = lambda x: analyzer.polarity_scores(x)

In [12]:
#dfNORTtextcleanEN
clean_df=dfNORTtextcleanEN

In [ ]:
# creating new column 'polarity' in clean_df
clean_df['polarity'] = clean_df['Newtext'].apply(pol)

In [ ]:
# checking that worked
clean_df.head(2)

In [ ]:
polaritydf=clean_df['polarity'].apply(pd.Series)
polaritydf

In [ ]:
df = pd.concat([clean_df, polaritydf], axis=1 )
df.head(2)

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
data = df['compound']
figsize=(20,10)
#N, bins, patches = ax.hist(data, bins=100, edgecolor='white', linewidth=1)
ax.set_ylabel("Number of tweets", fontsize=14)
ax.set_xlabel("Compound of tweets (negative <----> positive)", fontsize=14)
ax.set_title('Tweet (VADER) Compound visualisation',fontsize=18)
N, bins, bars = ax.hist(data, bins=100, color="skyblue", edgecolor='white', linewidth=1)
for bar in bars:
    if bar.get_x() < 0:
        bar.set_facecolor("lightcoral")
plt.savefig(folder+'VADERCompound'+file+'.pdf')